# 02. 프롬프트(Prompt)
- 프롬프트 단계는 검색기에서 검색된 문서들을 바탕으로 **언어 모델이 사용할 질문이나 명령을 생성하는 과정**
- 이 단계는 검색된 정보를 바탕으로 최종 사용자의 질문에 가장 잘 대응할 수 있는 응답을 생성하기 위해 필수적인 단계

<br>

### 프롬프트의 필요성
- **문맥(Context) 설정**: 프롬프트는 언어 모델이 특정 문맥에서 작동하도록 설정하는 역할
  - 이를 통해 모델은 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성

- **정보 통합** : 여러 문서에서 검색된 정보는 서로 다른 관점이나 내용을 포함할 수 있기에, 프롬프트 단계에서 이러한 정보를 통합하고, 모델이 이를 효율적으로 활용할 수 있는 형식으로 조정
- **응답 품질 향상** : 질문에 대한 모델의 응답 품질은 프롬프트의 구성에 크게 의존. 잘 구성된 프롬프트는 모델이 보다 정확하고 유용한 정보를 제공

<br>

### RAG 프롬프트 구조
- 지시사항(Instruction)
- 질문(사용자 입력 질문)
- 문맥(검색된 정보)

```md
당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요. 
만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. 
한국어로 대답하세요.

#Question: 
{이곳에 사용자가 입력한 질문이 삽입됩니다}

#Context: 
{이곳에 검색된 정보가 삽입됩니다}

```

<br>

### 프롬프트의 중요성
- 이 단계를 통해 언어 모델은 사용자의 질문에 대해 최적화된 방식으로 응답을 생성할 수 있으며, 시스템 전체의 성능과 사용자 만족도에 직접적인 영향 
- 프롬프트가 잘 구성되어 있지 않으면, 모델이 비효율적으로 작동할 수 있으며, 결과적으로 사용자의 요구에 부응하지 못하는 응답을 생성할 가능성이 높아짐

<br>

## 02-01. 프롬프트(Prompt)

<br>

### `PromptTemplate`

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

- LLM 객체를 정의

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI(model='gpt-4o-mini')

<br>

### 방법 1. `from_template()` 메소드를 사용하여 `PromptTemplate` 객체 생성
- 치환될 변수를 `{ 변수 }` 로 묶어서 템플릿을 정의

In [5]:
from langchain_core.prompts import PromptTemplate

In [6]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

- chain 생성

In [7]:
chain = prompt | llm

In [8]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

<br>

### 방법 2. `PromptTemplate` 객체 생성과 동시에 `prompt` 생성
* 추가 유효성 검사를 위해 `input_variables` 를 명시적으로 지정
* 이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생

In [ ]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국" 
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [17]:
prompt_partial = prompt.partial(country2="캐나다")

chain = prompt_partial | llm

In [18]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [19]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

<br>

#### `partial_variables`: 부분 변수 채움
- **`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것**
  - 이 사용 사례는 항상 공통된 방식으로 가져오고 싶은 변수 가 있는 경우 (예: 날짜/시간)
  - 항상 현재 날짜가 표시되기를 원하는 프롬프트가 있을 떄, 항상 현재 날짜를 반환하는 함수 를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리

In [6]:
from datetime import datetime

- 날짜를 반환하는 함수 정의

In [7]:
def get_today():
    return datetime.now().strftime("%B %d")

In [22]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today
    },
)

prompt.format(n=3)

'오늘의 날짜는 October 01 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [23]:
chain = prompt | llm

In [24]:
print(chain.invoke(3).content)

10월 1일에 생일인 유명인 3명은 다음과 같습니다:

1. **일라 스튜어트 (Ila St. John)** - 1937년 10월 1일
2. **브래드 피트 (Brad Pitt)** - 1963년 10월 1일
3. **제이크 질렌할 (Jake Gyllenhaal)** - 1980년 10월 1일

이 외에도 여러 유명인들이 있지만, 이 세 명이 특히 잘 알려져 있습니다.


In [26]:
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

오늘인 1월 2일에 생일인 유명인 3명은 다음과 같습니다:

1. **Isaac Asimov** - 1920년 1월 2일
2. **J. R. R. Tolkien** - 1892년 1월 2일
3. **Dax Shepard** - 1975년 1월 2일

이 외에도 1월 2일에 태어난 많은 유명인들이 있습니다!


<br>

### 파일로부터 template 읽어오기

In [8]:
from langchain_core.prompts import load_prompt

<br>

- `fruit_color.yaml`

```yaml
_type: prompt
input_variables:
  - fruit
template: |
  {fruit}의 색깔이 뭐야?
```

In [9]:
prompt = load_prompt("prompts/fruit_color.yaml")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [10]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [11]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:


<br>

### `ChatPromptTemplate`
- `ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용
- 메시지는 튜플(tuple) 형식으로 구성하며, (role, message) 로 구성하여 리스트로 생성할 수 있음
- `role` 
  - `"system"` : 시스템 설정 메시지. 주로 전역설정과 관련된 프롬프트. 
  - `"human"` : 사용자 입력 메시지
  - `"ai"`: AI 의 답변 메시지

In [12]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [14]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [37]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [38]:
llm.invoke(messages).content

'제 이름은 테디입니다! 어떻게 도와드릴까요?'

- 체인을 생성

In [39]:
chain = chat_template | llm

In [40]:
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Teddy입니다! 당신과 이야기하게 되어 기쁩니다. 어떤 도움이 필요하신가요?'

<br>

### `MessagePlaceholder`
- LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder` 를 제공
  - **메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용**

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [42]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

- `conversation` 대화목록을 나중에 추가하고자 할 때 `MessagesPlaceholder` 를 사용

In [43]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [44]:
chain = chat_prompt | llm | StrOutputParser()

In [45]:
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'새로 입사한 테디, 반가움.'

<br>

<hr>

<br>

## 02-02 퓨샷 프롬프트(FewShotPromptTemplate)
- Few-Shot Prompting : 모델에게 "예시 몇 개"를 같이 보여주면서, 앞으로의 입력도 그 패턴대로 답변하도록 유도하는 프롬프트 기법
  - 예시가 전혀 없는 것을 Zero-Shot, 하나만 있는 것을 One-Shot, 몇 개 있는 것을 Few-Shot


| 이유                  | 설명                                     |
| ------------------- | -------------------------------------- |
| **출력 패턴 통제**      | 원하는 형식(예: JSON, 문장 구조, 말투 등)을 쉽게 유도 가능 |
| **작업 이해 향상**      | 예시를 통해 문제의 의미나 범위를 명확히 인식              |
| **작은 데이터로 튜닝 효과** | 별도 모델 학습 없이도 “작은 학습 효과”를 얻음            |
| **제로샷보다 정확도↑**    | 예시 덕분에 오답률, 불필요한 말줄임 등이 줄어듦            |



<br>

### `FewShotPromptTemplate`

In [ ]:
llm = ChatOpenAI(
    temperature=0,  # 창의성
    model_name="gpt-4o-mini",  # 모델명
)

question = "대한민국의 수도는 뭐야?"

answer = llm.stream(question)

for chunk in answer:
    if chunk:
        print(chunk.content, end='', flush=True)

대한민국의 수도는 서울입니다.

In [17]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [18]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
            """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
            """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 율곡 이이의 어머니는 누구인가요?
            중간 답변: 율곡 이이의 어머니는 신사임당입니다.
            추가 질문: 신사임당은 언제 태어났나요?
            중간 답변: 신사임당은 1504년에 태어났습니다.
            추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
            중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
            최종 답변은: 연산군
            """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 올드보이의 감독은 누구인가요?
            중간 답변: 올드보이의 감독은 박찬욱입니다.
            추가 질문: 박찬욱은 어느 나라 출신인가요?
            중간 답변: 박찬욱은 대한민국 출신입니다.
            추가 질문: 기생충의 감독은 누구인가요?
            중간 답변: 기생충의 감독은 봉준호입니다.
            추가 질문: 봉준호는 어느 나라 출신인가요?
            중간 답변: 봉준호는 대한민국 출신입니다.
            최종 답변은: 예
            """,
    },
]

In [19]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
            


In [20]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [21]:
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
            

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
            

Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 율곡 이이의 어머니는 누구인가요?
            중간 답변: 율곡 이이의 어머니는 신사임당입니다.
            추가 질문: 신사임당은 언제 태어났나요?
            중간 답변: 신사임당은 1504년에 태어났습니다.
            추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
            중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
            최종 답변은: 연산군
            

Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 올드보이의 감독은 누구인가요?
        

In [22]:
answer = llm.stream(final_prompt)

for chunk in answer:
    if chunk:
        print(chunk.content, end='', flush=True)

이 질문에 추가 질문이 필요한가요: 예.  
            추가 질문: Google은 언제 창립되었나요?  
            중간 답변: Google은 1998년에 창립되었습니다.  
            추가 질문: Bill Gates는 언제 태어났나요?  
            중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
            추가 질문: 1998년 당시 Bill Gates의 나이는 몇 살이었나요?  
            중간 답변: 1998년에는 Bill Gates가 42세였습니다.  
            최종 답변은: 42세

<br>

### `Example Selector`
- 예제가 많은 경우 프롬프트에 포함할 예제를 선택 $\rightarrow$ `Example Selector` 는 이 작업을 담당하는 클래스
- 사용자의 입력과 의미적으로 가장 관련성 높은 예시(few-shot example)를 자동으로 선택하여 프롬프트에 포함시키는 도구 또는 메커니즘

In [23]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [24]:
# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

In [26]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # 선택 가능한 예시 목록
    OpenAIEmbeddings(), # 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스
    Chroma, # 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스
    k=1, # 생성할 예시의 수
)

In [27]:
selected_examples = example_selector.select_examples({"question": question})

In [ ]:
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
print(f"입력에 가장 유사한 예시:\n{question}\n")

for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

입력에 가장 유사한 예시:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
            


In [30]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
            

Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


- `ExampleSelector` 를 사용하여 `FewShotPromptTemplate` 을 생성

In [31]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

chain = prompt | llm

In [32]:
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)

for chunk in answer:
    if chunk:
        print(chunk.content, end='', flush=True)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 몇 년도에 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 Bill Gates는 몇 살이었나요?  
중간 답변: 1998년에는 Bill Gates가 43세였습니다.  
최종 답변은: 43세

<br>

### `FewShotChatMessagePromptTemplate`

In [ ]:
examples = [
   {
      "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
      "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
      "answer": """
      회의록: XYZ 회사 마케팅 전략 회의
      일시: 2023년 12월 25일
      장소: XYZ 회사 회의실
      참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

      1. 개회
         - 회의는 김수진 팀장의 개회사로 시작됨.
         - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

      2. 시장 동향 개요 (김수진)
         - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
         - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

      3. 디지털 마케팅 전략 (박지민)
         - 박지민은 디지털 마케팅 전략에 대해 발표.
         - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

      4. 소셜 미디어 캠페인 (이준호)
         - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
         - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

      5. 종합 논의
         - 팀원들 간의 아이디어 공유 및 토론.
         - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

      6. 마무리
         - 다음 회의 날짜 및 시간 확정.
         - 회의록 정리 및 배포는 박지민 담당.
      """,
   },
   {
      "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
      "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
      "answer": """
      문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

      - 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
      - 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
      - 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
      - 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
      - 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

      이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
      """,
   },
   {
      "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
      "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
      "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
   },
]

In [34]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [35]:
chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

In [37]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

In [38]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    chroma,
    k=1,
)

In [39]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

- fewshot 예제와 example selector를 사용하여 유사한 예제 1개를 선택

In [40]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'answer': '\n         회의록: XYZ 회사 마케팅 전략 회의\n         일시: 2023년 12월 25일\n         장소: XYZ 회사 회의실\n         참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n         1. 개회\n            - 회의는 김수진 팀장의 개회사로 시작됨.\n            - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n         2. 시장 동향 개요 (김수진)\n            - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n            - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n         3. 디지털 마케팅 전략 (박지민)\n            - 박지민은 디지털 마케팅 전략에 대해 발표.\n            - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n         4. 소셜 미디어 캠페인 (이준호)\n            - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n            - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n         5. 종합 논의\n 

- 가장 유사한 예제를 기반으로한 답변

In [41]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [42]:
chain = final_prompt | llm
answer = chain.stream(question)
for chunk in answer:
    if chunk:
        print(chunk.content, end='', flush=True)

**회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의**  
**일시:** 2023년 12월 26일  
**장소:** ABC 기술 회사 회의실  
**참석자:** 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)  

---

### 1. 개회
- 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.  
- 회의의 주요 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤 계획 수립.

### 2. 진행 상황 보고
- **최현수 (프로젝트 매니저)**
  - 전반적인 프로젝트 일정 및 현황 점검.
  - 진행 중인 작업의 타임라인에 대해 설명.
  
- **황지연 (주요 개발자)**
  - 개발 진행 상황에 대한 업데이트 제공.
  - 현재 완료된 기능과 남은 개발 작업에 대해 보고.

- **김태영 (UI/UX 디자이너)**
  - 디자인 작업 진행 상황 및 피드백 수렴 결과 공유.
  - 사용자 테스트 결과에 대한 논의.

### 3. 다가오는 마일스톤 계획
- 각 팀원은 다음 주까지의 구체적 목표 설정.
- 마일스톤 달성을 위한 필요한 리소스 및 지원 사항 검토.

### 4. 종합 논의
- 팀원 간의 아이디어 및 피드백 공유.
- 발생 가능한 문제점 및 해결 방안 논의.
  
### 5. 마무리
- 다음 회의 일정 및 안건 확정.
- 회의록 작성 및 배포는 최현수 담당.  

---

**회의 종료:** 오후 4시 30분.  
**작성자:** 최현수 (프로젝트 매니저)

<br>

### `Example Selector` 의 유사도 검색 문제 해결
- 유사도 계산시 `instruction` 과 `input` 을 사용하고 있지만, `instruction` 만 사용하여 검색시 제대로된 유사도 결과가 나오지 않음

    $\rightarrow$ 커스텀 유사도 계산을 위한 클래스를 정의
    
<br>

- 잘못 검색된 결과의 예시

In [43]:
question = {
    "instruction": "회의록을 작성해 주세요",
}

example_selector.select_examples(question)

[{'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

- 커스텀 하지 않은 기본 예제 선택기를 사용했을 때 결과

In [44]:
example_selector.select_examples({"instruction": "다음 문장을 요약해 주세요"})

[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

- 커스텀 예제 선택기 생성

In [ ]:
from langchain_teddynote.prompts import CustomExampleSelector

In [46]:
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

- 커스텀 예제 선택기를 사용했을 때 결과


In [47]:
custom_selector.select_examples({"instruction": "다음 문장을 교정 작성해 주세요"})

[{'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.'}]

In [48]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

In [ ]:
custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  
)

In [50]:
custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [51]:
chain = custom_prompt | llm

In [52]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

for chunk in chain.stream(question):
    
    print(chunk.content, end='', flush=True)

### 회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의

**일시**: 2023년 12월 26일  
**장소**: ABC 기술 회사 회의실  
**참석자**: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

---

**1. 개회**  
- 최현수 프로젝트 매니저의 개회사로 회의 시작.  
- 회의의 주요 목적: 새로운 모바일 애플리케이션 프로젝트의 진행 상황 검토 및 다가오는 마일스톤 계획 수립.

**2. 진행 상황 보고**  
- **최현수 (프로젝트 매니저)**  
  - 전체 프로젝트 일정 및 현재 상태를 개괄.  
  - 각 팀원에게 진행 상황 피드백 요청.

- **황지연 (주요 개발자)**  
  - 백엔드 개발 진행 상황 보고.  
  - 주요 기능 구현 완료 상황 및 문제점에 대해 설명.  
  - 데이터베이스 구조 변경 사항 논의.

- **김태영 (UI/UX 디자이너)**  
  - 현재 UI/UX 디자인 초안 진행 상황 공유.  
  - 사용자 피드백 반영 과정 및 수정 작업에 대한 업데이트.  
  - 몇 가지 디자인 요소에 대한 제안과 논의.

**3. 마일스톤 계획**  
- 팀원들은 다음 주까지 달성할 목표에 대해 협의.  
- 각자 역할에 따라 세부 목표 및 마감일 설정:  
  - 황지연은 기능 테스트 및 버그 수정 완료.  
  - 김태영은 최종 디자인 리뷰 및 피드백 반영 완료.

**4. 이슈 및 논의**  
- 프로젝트 일정과 관련된 잠재적 문제점 논의.  
- 필요한 추가 자원이나 지원 요청 사항 정리.

**5. 마무리**  
- 다음 회의 일정 설정: 2024년 1월 2일.  
- 최현수의 회의 진행에 대한 감사 인사로 회의 종료.  
- 회의록 정리는 김태영이 담당하기로 함.

--- 

회의 종료 시각: 오후 4시 30분

In [53]:
question = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}

for chunk in chain.stream(question):
    
    print(chunk.content, end='', flush=True)

문서 요약: 2023년 글로벌 경제 전망 보고서

- **현재 상태**: 세계 경제의 현재 상황을 평가하고, 주요 국가들의 경제 성장률을 분석.
- **경제 성장률**: 주요 국가들(미국, 중국, 유럽연합 등)의 경제 성장률 변화를 통계적으로 제시.
- **글로벌 무역 동향**: 최근의 글로벌 무역 트렌드 및 주요 흐름을 설명, 특히 수출입 변화와 새로운 무역 협정 등을 분석.
- **미래 예측**: 2024년을 포함한 다가오는 해에 대한 경제 전망을 다양한 시나리오로 제시.
- **영향 요인**: 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 잠재적 영향을 분석하며, 이러한 요인들이 경제 성장에 미치는 장단기적 효과를 평가.

이 보고서는 글로벌 경제의 복잡한 변화와 전망에 대한 종합적인 분석을 제공하며, 앞으로의 경제 정책 수립을 위한 기초 자료로 활용될 수 있습니다.

In [54]:
question = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}

for chunk in chain.stream(question):
   
    print(chunk.content, end='', flush=True)

회사는 올해 매출이 증가할 것으로 예상하고 있으며, 새로운 전략이 잘 작동하고 있다.

<br>

<hr>

<br>

## 02-03. LangChain Hub

<br>

### Hub로부터 Prompt 받아오기

<br>

#### Prompt Hub
- LangChain에는 GitHub처럼 “공유 가능한 프롬프트 저장소” 개념이 존재
    - 주소: https://smith.langchain.com/hub (공식 Prompt Hub)

In [9]:
from langchain import hub

In [10]:
prompt = hub.pull("rlm/rag-prompt")

In [11]:
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


- 특정 버전의 프롬프트를 가져오려면 버전 해시를 지정

In [12]:
prompt = hub.pull("rlm/rag-prompt:50442af1")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

<br>

### Prompt Hub 에 프롬프트 등록

In [23]:
from langsmith import Client

In [14]:
prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)

prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})])

- 계정 확인

In [24]:
client = Client()

In [21]:
client.create_prompt(
    prompt_identifier="simple-summary-korean",
    description="한국어 요약용 프롬프트",
    is_public=False
)

Prompt(repo_handle='simple-summary-korean', description='한국어 요약용 프롬프트', readme='', id='768b6c99-b544-4a2a-bcdc-41c4ea196669', tenant_id='5ede379f-7d77-4421-a6dc-44285dc72ab3', created_at=datetime.datetime(2025, 10, 1, 8, 4, 11, 93478), updated_at=datetime.datetime(2025, 10, 1, 8, 4, 11, 93478), is_public=False, is_archived=False, tags=[], original_repo_id=None, upstream_repo_id=None, owner=None, full_name='simple-summary-korean', num_likes=0, num_downloads=0, num_views=0, liked_by_auth_user=False, last_commit_hash=None, num_commits=0, original_repo_full_name=None, upstream_repo_full_name=None)

In [22]:
client.push_prompt(
    "simple-summary-korean",
    object=prompt
)

'https://smith.langchain.com/prompts/simple-summary-korean/de93adb6?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

<hr>

<br>

## 02-04. 개인화된 프롬프트

<br>

#### 요약: Stuff Documents : 문장을 핵심만 이모지와 함께 불릿 포인트로 요약하는 요청

In [25]:
prompt_title = "summary-stuff-documents"

- 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)


In [27]:
prompt_template = """Please summarize the sentence according to the following REQUEST.
    REQUEST:
    1. Summarize the main points in bullet points.
    2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
    3. Use various emojis to make the summary more interesting.
    4. DO NOT include any unnecessary information.

    CONTEXT:
    {context}

    SUMMARY:"
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Please summarize the sentence according to the following REQUEST.\n    REQUEST:\n    1. Summarize the main points in bullet points.\n    2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n    3. Use various emojis to make the summary more interesting.\n    4. DO NOT include any unnecessary information.\n\n    CONTEXT:\n    {context}\n\n    SUMMARY:"\n')

In [28]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/summary-stuff-documents/af418379?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

#### Map Prompt
- 문서의 주요 주제를 핵심만 번호 목록 형태로 요약하는 요청

In [29]:
prompt_title = "map-prompt"

In [30]:
prompt_template = """You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.
    Please provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format. 
    The summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format. 
    Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition. 
    The length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.

    GIVEN DOCUMENTS:
    {docs}

    FORMAT:
    1. main theme 1
    2. main theme 2
    3. main theme 3
    ...

    CAUTION:
    - DO NOT list more than 5 main themes.

    Helpful Answer:
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.\n    Please provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format. \n    The summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format. \n    Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition. \n    The length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.\n\n    GIVEN DOCUMENTS:\n    {docs}\n\n    FORMAT:\n    1. main theme 1\n    2. main theme 2\n    3. main theme 3\n    ...\n\n    CAUTION:\n    - DO NOT list more than 5 main themes.\n\n   

In [31]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/map-prompt/291cf33a?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

#### Reduce Prompt
- 여러 요약 목록에서 핵심 인사이트 10가지를 추출해 한국어로 요약하는 요청

In [33]:
prompt_title = "reduce-prompt"

In [34]:
prompt_template = """You are a helpful expert in summary writing.
    You are given numbered lists of summaries.
    Extract top 10 most important insights from the summaries.
    Then, write a summary of the insights in KOREAN.

    LIST OF SUMMARIES:
    {doc_summaries}

    Helpful Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['doc_summaries'], input_types={}, partial_variables={}, template='You are a helpful expert in summary writing.\n    You are given numbered lists of summaries.\n    Extract top 10 most important insights from the summaries.\n    Then, write a summary of the insights in KOREAN.\n\n    LIST OF SUMMARIES:\n    {doc_summaries}\n\n    Helpful Answer:\n')

In [35]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/reduce-prompt/1a20d9d3?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

- 이전에 요약된 문장을 기반으로 핵심 내용을 중요도 순서대로 이모지와 함께 불릿 포인트로 한국어 요약하는 요청

In [36]:
prompt_title = "chain-of-density-reduce-korean"

In [37]:
prompt_template = """You are a helpful expert in summary writing. You are given lists of summaries.
    Please sum up previously summarized sentences according to the following REQUEST.
    REQUEST:
    1. Summarize the main points in bullet points in KOREAN.
    2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
    3. Use various emojis to make the summary more interesting.
    4. MOST IMPORTANT points should be organized at the top of the list.
    5. DO NOT include any unnecessary information.

    LIST OF SUMMARIES:
    {doc_summaries}

    Helpful Answer: 
"""

In [38]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/chain-of-density-reduce-korean/1a20d9d3?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

#### Metadata Tagger
- 제품 리뷰에서 키감·키음·사용경험·충전·디자인 등 핵심 속성을 추출하고 감성 톤과 1~5점 평점을 판별한 뒤 디자인 평가·만족/개선 사항·키워드와 함께 JSON으로 구조화해 요약하는 요청

In [39]:
prompt_title = "metadata-tagger"

In [ ]:
prompt_template = """
Given the following product review, conduct a comprehensive analysis to extract key aspects mentioned by the customer, with a focus on evaluating the product's design and distinguishing between positive aspects and areas for improvement. 
Identify primary features or attributes of the product that the customer appreciated or highlighted, specifically looking for mentions related to the feel of the keys, sound produced by the keys, overall user experience, charging aspect, and the design of the product, etc. 
Assess the overall tone of the review (positive, neutral, or negative) based on the sentiment expressed about these attributes. 
Additionally, provide a detailed evaluation of the design, outline the positive aspects that the customer enjoyed, and note any areas of improvement or disappointment mentioned. 
Extract the customer's rating of the product on a scale of 1 to 5, as indicated at the beginning of the review. 
Summarize your findings in a structured JSON format, including an array of keywords, evaluations for design, satisfaction points, improvement areas, the assessed tone, and the numerical rating. 

INPUT:
{input}
"""

In [41]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/metadata-tagger/1a20d9d3?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

#### Chain of Density 요약
- 주어진 기사에서 누락된 핵심 엔티티를 단계적으로 찾아내고, 매번 동일한 길이의 요약문을 더 압축·정보밀도 높게 갱신해 나가는 반복적 요약 생성 요청

In [42]:
prompt_title = "chain-of-density-korean"

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
    You will generate increasingly concise, entity-dense summaries of the above article. 

    Repeat the following 2 steps 5 times. 

    Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
    Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

    A missing entity is:
    - relevant to the main story, 
    - specific yet concise (100 words or fewer), 
    - novel (not in the previous summary), 
    - faithful (present in the article), 
    - anywhere (can be located anywhere in the article).

    Guidelines:

    - The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
    - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
    - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
    - The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
    - Missing entities can appear anywhere in the new summary.
    - Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

    Remember, use the exact same number of words for each summary.
    Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
    Use only KOREAN language to reply.
    """
)

In [44]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/chain-of-density-korean/2d444953?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

#### Chain of Density (Korean) - 2
- 기사에서 누락된 핵심 엔티티를 단계적으로 식별하고 이를 포함해 동일한 길이의 요약문을 점점 더 압축·고밀도로 갱신하는 반복 요약 생성 요청

In [45]:
prompt_title = "chain-of-density-map-korean"

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
    You will generate increasingly concise, entity-dense summaries of the above article. 

    Repeat the following 2 steps 3 times. 

    Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
    Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

    A missing entity is:
    - relevant to the main story, 
    - specific yet concise (100 words or fewer), 
    - novel (not in the previous summary), 
    - faithful (present in the article), 
    - anywhere (can be located anywhere in the article).

    Guidelines:

    - The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
    - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
    - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
    - The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
    - Missing entities can appear anywhere in the new summary.
    - Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

    Remember, use the exact same number of words for each summary.
    Answer "Missing Entities" and "Denser_Summary" as in TEXT format.
    Use only KOREAN language to reply.
    """
)

In [47]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/chain-of-density-map-korean/0b372530?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'

<br>

#### RAG 문서 프롬프트

In [48]:
prompt_title = "rag-prompt-korean"

In [49]:
system = """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."""

human = """#Question: 
{question} 

#Context: 
{context} 

#Answer:"""

In [50]:
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

In [51]:
client.create_prompt(
    prompt_identifier=prompt_title,
    is_public=False
)

client.push_prompt(
    prompt_title,
    object=prompt
)

'https://smith.langchain.com/prompts/rag-prompt-korean/cbfb554c?organizationId=5ede379f-7d77-4421-a6dc-44285dc72ab3'